In [1]:
import pandas as pd
import pickle
import shapely

In [91]:
#to_correct = 'Datasets/data_FEC_NATIONALPOLL_2004_2018.p'
to_correct = 'Datasets/data_FEC_NATIONALPOLL_DEMOGRAPHICS_2010_2018.p'

overlap = pickle.load(open('Datasets/all_overlap_data.p', 'rb'))
dataset = pickle.load(open(to_correct, 'rb'))
dataset.shape

(2126, 42)

In [92]:
dataset[dataset['year'] == 2012].shape

(423, 42)

In [93]:
#drop redistricted districts
new_dataset_name = to_correct[:-2] + '_REDISTRICTDROP.p'
border_change = overlap['border_change']
border_change = border_change[~border_change.isnull()]
border_change = border_change[border_change == 1] #1: border changed, 0: border not changed
for cand_index in dataset.index.values:
    if cand_index in border_change.index:
        dataset = dataset[dataset.index != cand_index]
dataset.shape

(1749, 42)

In [94]:
dataset[dataset['year'] == 2012].shape

(99, 42)

In [95]:
centroids = overlap['centroid']
for cand_index in dataset.index.values:
    lon, lat = centroids[centroids.index == cand_index].values[0][0]
    dataset.loc[dataset.index == cand_index, 'longitude'] = lon
    dataset.loc[dataset.index == cand_index, 'latitude'] = lat
dataset.shape, dataset.columns

((1749, 44),
 Index(['district', 'state', 'year', 'party', 'candidatevotes', 'totalvotes',
        'candidate', 'national_poll', 'national_poll_prev',
        'national_poll_delta_subtract', 'national_poll_delta_divide',
        'previous_party', 'dem_win_margin_prev', 'rep_win_margin_prev',
        'margin_signed_minus_prev', 'margin_signed_divide_prev',
        'margin_unsigned_minus_prev', 'margin_unsigned_divide_prev',
        'dem_win_prev', 'rep_win_prev', 'dem_win', 'rep_win', 'female_pct',
        'age18_24_pct', 'age25_34_pct', 'median_age', 'unmarried_partner_pct',
        'male_living_alone_pct', 'bachelors_deg_or_higher_pct',
        'past_year_births_to_unmarried_women_pct', 'civilian_veteran_pct',
        'live_same_house_past_year_pct',
        'foreign_born_proportion_from_LatinAmerica',
        'speak_other_language_at_home_pct', 'labor_force_unemployed_pct',
        'public_transit_commuter_pct', 'no_health_insurance_pct',
        'poverty_rate_pct', 'median_housing_v

In [98]:
dataset[dataset['year']!=2018].isnull().values.any()

False

In [96]:
pickle.dump(dataset, open(new_dataset_name, 'wb'))